# P3 Wrangle OpenStreetMap Data - SQL - Ahmad Takatkah


----------

## 1. Map Selection


I've selected the Mapzen Metro Extract for **Amman, Jordan** [[1]](https://mapzen.com/data/metro-extracts/metro/amman_jordan/), my home city.  


Amman (English pronunciation: /ɑːˈmɑːn/; Arabic: عمّان) is the capital and most populous city of Jordan, and the country's economic, political and cultural centre. The city has a population of 4,007,526 and a land area of 1,680 square kilometres (648.7 sq mi). Today, Amman is considered to be among the most liberal and westernized Arab cities. It is a major tourist destination in the region, particularly among Arab and European tourists. [[2]](https://en.wikipedia.org/wiki/Amman)



<img style="float: center;" src="Amman_Jordan_map.png" width="500" height="500">

--------

## 2. Data Overview

The uncompressed OSM XML file details: 


    name: amman_jordan.osm
    size: 57.9 MB


I created a sample taking every 50th element from the original dataset by running the file named: `create_sample.py` in the submitted folder (relations and bounds were ignored in the sample). The sample is needed for the purpose of initial auditing, looking at the data in a smaller file, and (later on) schema validation (using the file named: `schema.py` in the submitted folder). 

The sample OSM XML file details:


    name: amman_jordan_sample.osm
    size: 1.2 MB

-----------

To get an initial idea of the number and names of tags in the dataset, I used the etree library to iterate through the file, find tags, and add them to an integer default dictionary, this was done by running the file named: `overview.py` in the submitted folder. 

Here are the names and numbers of all the tags in the **full dataset**: 


    'node': 260,803 
    'nd': 336,735 
    'bounds': 1 
    'member': 395
    'tag': 114,563
    'osm': 1 
    'way': 50,374 
    'relation': 100
    
 
To get even a better idea of the raw data structure, let's count the highest_level tags in the **full dataset**: 
 
Here are the results:

    'node': 260803 
    'way': 50374
    'relation': 100
    'bounds': 1


Names and numbers of all the tags in the **sample file**: 


    'node': 5217 
    'tag': 2274 
    'nd': 6842 
    'response': 1 
    'way': 1007
    
    
Names and numbers of primary tags in the **sample file**: 


    'node': 5217 
    'way': 1007


--------

Let's now count the number of users contributing to the map: 

    unique users : 511 

Later on in this project, and after creating the SQL database, we will investigate what kinds of users contributed to the map, and who contributed the most to the map data. 


-------

## 3. Data Auditing 

By looking at the data in the sample file, we see that the tags named "tag" inside a "node" or "way" element has keys' names with colons and underscore symbols, and some are in Arabic Language.   

Let's explore the **full dataset** in more details by inspecting how many of the keys' names in the tags named "tag" have the following cases:
- any tag key with **Arabic** characters 
- all lower case letters with no special chararcters except for "_"
- all lower case letters with a colon ":"
- any tag key with problem characters
- other cases 

The following results were generated by running the file names: `audit.py` in the submitted folder. 

The result from the **full dataset** is as follows: 

    'arabic': 4
    'lower': 109668
    'lower_colon': 4856
    'problemchars': 20
    'other': 15

There are only 4 Arabic key names, which we can replace later on with their English translation as follows:

    "اسماء": "name",
    "معرض": "shop",
    "مسجد": "mosque",

There are only 20 key names with problem characters, and can be ignored. 


There are 15 "others" and those are key names with either capital letters or numbers within the key name, which we will keep. Examples:

    other: fuel:octane_95
    other: fuel:HGV_diesel
    other: name_1
    other: name_2




But we know by looking at the sample data that we have some Arabic values (mainly names of places and streets), so let's see how many Arabic values do we have in the data:

    'arabic': 3620 
    'other': 110943

This seems to be representing 3.26% of the whole dataset, but in reality it's much less than this because most of the names are provided in mutilple langaues, so those names are also provided in English. 


Example:

```xml
    <tag k="name" v="وادي صويلح"/>
    <tag k="is_in" v="Jordan, الأردن"/>
    <tag k="place" v="locality"/>
    <tag k="name:ar" v="وادي صويلح"/>
    <tag k="name:de" v="Wadi Suweilih"/>
    <tag k="name:en" v="Wadi Suweilih"/>
    <tag k="name:fr" v="Wadi Suweilih"/>
    <tag k="int_name" v="Wādī Ṣuweiliḥ; Wadi Suweilih"/>```
    


-------

By looking at the sample data, we also notice that 

English street names are: 

    "Road", "Street", "Highway", "Autostrad"

Arabic street names are:

    "شارع" , "طريق"
    
let's audit the full dataset and see if there are any other names for streets and cities, what are they, and if we need to change them to standardize the data. 


The results don't show any issues for Arabic street names. However, there are some inconsistencies in English street names, English cities names, and Arabic city names.


Later on in the data cleaning section of this project, we will be standardizing the names as follows: 

Street Names (English):

    "Sreet" ->  "Street",
    "sreet" -> "Street",
    "street" -> "Street",
    "St" -> "Street",
    "St." -> "Street",
    
City Names (English):

    "Jabal Amman" -> "Amman",
    "ammn" -> "Amman",
    "amman" -> "Amman",
    "Shmeisani" -> "Amman",
    "Al-Salt" -> "Al Salt",

City Names (Arabic): 

    "عمان" -> "Amman",
    "عمان‎" -> "Amman",
    "Amman Jordan عمان الأردن" -> "Amman",
    "Amman - Al Bayader عمان \ البيادر" -> "Amman",
    "السخنة" -> "Al Sukhnah",
    "الزرقاء" -> "Al Zarqa",
    "الزرقاء‎" -> "Amman",
    "الرصيفة" -> "Al Zarqa",



## 4. Data Cleaning

By running the file named: `process.py` in the submitted folder, I am cleaning the dataset based on the above audit, and creating five .csv files to store the cleaned data following the guidelines provided by Udacity (listed in process.py as comments)


The output .csv file sizes are as follows: 

    nodes.csv        : 20.6 MB
    nodes_tags.csv   : 0.618 MB
    ways.csv         : 2.8 MB
    ways_nodes.csv   : 8.1 MB
    ways_tags.csv    : 3.6 MB

------

## 5. Creating the SQL Database

Now, we're ready to create our SQLite database. Below is the code to initiate a new database called: `amman.db`. Then iterate through the .csv files to convert them to Pandas dataframes, and then to tables in the SQLite database with the same names of the files.  




In [1]:
import pandas as pd
from sqlalchemy import create_engine

chunksize = 1000000
amman_db = create_engine('sqlite:///amman.db')

# Using only one chunk as the dataset is not too big to loop through the chunks and append them
def create_table(filename, tablename): 
    for df in pd.read_csv(filename, chunksize=chunksize, encoding='utf-8', iterator=True):
        df.to_sql(tablename, amman_db, index=False, if_exists="replace")

create_table('nodes.csv', 'nodes')
create_table('nodes_tags.csv', 'nodes_tags')
create_table('ways.csv', 'ways')
create_table('ways_nodes.csv', 'ways_nodes')
create_table('ways_tags.csv', 'ways_tags')


## 6. Data Exploration 

Now, let's establish a connection with our new cleaned database and start querying it for some basic stats. 



------

In [2]:
# Opening a connection with the database

import sqlite3

db = sqlite3.connect("amman.db")
c = db.cursor() 


In [ ]:
# Initial queries to explore the dataset

# Number of nodes  
query = '''
    SELECT 
        COUNT(*)
    FROM nodes 
    ;'''
c.execute(query)
print "Number of nodes: ", c.fetchall()[0][0]

# Number of ways
query = '''
    SELECT 
        COUNT(*)
    FROM ways 
    ;'''
c.execute(query)
print "Number of ways: ", c.fetchall()[0][0]

# Number of unique users
query = '''
    WITH 
    users AS
    (SELECT 
        uid 
    FROM nodes 
    UNION ALL 
    SELECT 
        uid 
    FROM ways)
    SELECT 
        COUNT(DISTINCT users.uid)
    FROM
        users
    ;'''
c.execute(query)
print "Number of unique users: ", c.fetchall()[0][0]



Here are the results from the initial queries:

    Number of nodes:  260803
    Number of ways:  50374
    Number of unique users:  509

We notice here that the numbers from the SQL query are less than the numbers at the very top of this notebook by only a few counts (1-3), which could be explained by removing the problem characters and by excluding "relations". 

------------



Now, let's use another approach to display the outputs utilizing Pandas dataframes. 

In the following code block, I am displaying the headers of all 5 tables with only 3 raws. 


In [4]:
from IPython.display import display

def show_head(query, table_name):
    df = pd.read_sql_query(query, db)
    print table_name
    display(df.head(3))

query = '''
    SELECT 
        *
    FROM nodes 
    ;'''
show_head(query, 'nodes')

query = '''
    SELECT 
        *
    FROM nodes_tags 
    ;'''
show_head(query, 'nodes_tags')

query = '''
    SELECT 
        *
    FROM ways 
    ;'''
show_head(query, 'ways')

query = '''
    SELECT 
        *
    FROM ways_tags 
    ;'''
show_head(query, 'ways_tags')

query = '''
    SELECT 
        *
    FROM ways_nodes 
    ;'''
show_head(query, 'ways_nodes')


nodes


,id,lat,lon,user,uid,version,changeset,timestamp
0,21111424,32.031163,35.902614,lyx,28237,2,10718328,2012-02-18T10:03:25Z
1,21111479,32.032615,35.894143,lyx,28237,3,10822474,2012-02-28T20:39:26Z
2,21111490,32.037124,35.891936,lyx,28237,4,10822474,2012-02-28T20:39:26Z


nodes_tags


,id,key,value,type
0,278183826,highway,traffic_signals,regular
1,331525265,amenity,fuel,regular
2,332576154,source,Bing,regular


ways


,id,user,uid,version,changeset,timestamp
0,23409730,Marc le Grand,2249743,31,43490143,2016-11-08T15:34:02Z
1,24290400,IknowJoseph,30587,17,31476695,2015-05-26T15:36:37Z
2,24474655,cabal,22726,3,12956944,2012-09-02T17:18:03Z


ways_tags


,id,key,value,type
0,23409730,oneway,yes,regular
1,23409730,highway,motorway_link,regular
2,23409730,maxspeed,50,regular


ways_nodes


,id,node_id,position
0,23409730,253445619,0
1,23409730,3791502933,1
2,23409730,331413061,2


**Tables mapping:**


To connect tags with their nodes:
    
    nodes.id = nodes_tags.id

To connect tags with their ways:

    ways.id  = ways_tags.id
    
To connect nodes_tags to their parent ways:

    ways_nodes.node_id  = nodes_tags.id
    
--------

Let's now see who the top 10 contributers to the map are and when did they contribute! 


In [5]:
def show_result(query, rows, description):
    df = pd.read_sql_query(query, db)
    print description
    display(df.head(rows))

# Top 10 contributers and their first and last contributions
query = '''
    WITH 
    users AS
    (SELECT 
        uid,
        user,
        timestamp
    FROM nodes 
    UNION ALL 
    SELECT 
        uid,
        user,
        timestamp
    FROM ways)
    SELECT 
        user,
        COUNT(DISTINCT users.timestamp) AS contributions,
        MIN(DATE(timestamp)) AS first_contribution,
        MAX(DATE(timestamp)) AS last_contribution
    FROM users
    GROUP BY 1
    ORDER BY 2 DESC
    ;'''
show_result(query, 10, 'Top 10 contributers and their first and last contributions')

Top 10 contributers and their first and last contributions


,user,contributions,first_contribution,last_contribution
0,lyx,16115,2010-04-28,2012-07-14
1,cabal,6487,2012-08-27,2013-01-14
2,Jamal H,870,2014-02-20,2017-02-21
3,toyotagps,780,2015-10-23,2016-06-23
4,mifrajoha,772,2016-01-21,2016-01-26
5,Big_Borz,672,2016-02-02,2016-08-11
6,Amjad Shahrour,503,2015-03-06,2015-08-03
7,Ori952,346,2012-03-17,2016-09-17
8,maen alsayed,275,2016-03-16,2016-06-10
9,Bogdan Basic,180,2015-12-07,2016-04-08


## 7. City Insights


Below are some interesting insights generated by SQL queries with titles of the table to describe the output: 



In [6]:
# Top 5 used languages in Amman
query = '''
    SELECT 
        key, 
        COUNT(*) 
    FROM nodes_tags 
    WHERE type = 'name'
    GROUP BY 1 
    ORDER BY COUNT(*) DESC 
    LIMIT 5
    ;'''
show_result(query, 5, 'Top 5 used languages in Amman')

Top 5 used languages in Amman


,key,COUNT(*)
0,en,1366
1,ar,1192
2,fr,790
3,de,772
4,ru,14


In [7]:
# Most common shop types
query = '''
    SELECT 
        value, 
        COUNT(*) 
    FROM nodes_tags 
    WHERE key = 'shop'
    GROUP BY 1 
    ORDER BY COUNT(*) DESC 
    LIMIT 10
    ;'''
show_result(query, 10, 'Most common shop types')

Most common shop types


,value,COUNT(*)
0,supermarket,76
1,convenience,30
2,bakery,25
3,car_repair,24
4,clothes,18
5,confectionery,17
6,mobile_phone,11
7,books,10
8,greengrocer,10
9,alcohol,9


In [8]:
# Top 10 banks 
query = '''
    WITH banks AS (
    SELECT 
        DISTINCT id AS bank_node_id
    FROM nodes_tags 
    WHERE value = 'bank'
    )
    SELECT 
        value, 
        COUNT(*) 
    FROM nodes_tags 
    INNER JOIN banks ON banks.bank_node_id = nodes_tags.id
    WHERE key = 'en' AND type = 'name'
    GROUP BY 1 
    ORDER BY COUNT(*) DESC 
    LIMIT 10
    ;'''
show_result(query, 10, 'Top 10 banks')

Top 10 banks


,value,COUNT(*)
0,Housing Bank for Trade and Finance,10
1,Jordan Kuwait Bank,6
2,Arab Bank,4
3,Al Etihad bank,2
4,Al Rajhi Bank,2
5,Audi bank,2
6,Cambridge school,2
7,Egyptian Arab Land Bank,2
8,Global investment house,2
9,IGI,2


In [9]:
# Questionable data entry 1

query = '''
    SELECT 
        *
    FROM nodes_tags 
    WHERE id = 4609270189
    ;'''
show_result(query, 10, 'Questionable data entry 1') 

Questionable data entry 1


,id,key,value,type
0,4609270189,amenity,bank,regular
1,4609270189,en,Cambridge school,name
2,4609270189,en,Cambridge school,name
3,4609270189,operator,dana Riyalat,regular
4,4609270189,phone,0799000523,regular


In [10]:
# Questionable data entry 2

query = '''
    SELECT 
        *
    FROM ways_tags 
    WHERE id = 245144085
    ;'''
show_result(query, 10, 'Questionable data entry 2') 

Questionable data entry 2


,id,key,value,type
0,245144085,name,Temple of Hercules;معبد هيراكليس,regular
1,245144085,ar,معبد هيراكليس,name
2,245144085,de,Herkulestempel,name
3,245144085,en,Temple of Hercules,name
4,245144085,historic,ruins,regular
5,245144085,wikidata,Q20423672,regular
6,245144085,wheelchair,no,regular


In [11]:
# Questionable data entry 3

query = '''
    SELECT 
        *
    FROM ways_tags 
    WHERE key = 'name'
    ;'''
show_result(query, 5, 'Questionable data entry 3')

Questionable data entry 3


,id,key,value,type
0,24639865,name,Prince El Hassan Street,regular
1,24939011,name,سد الملك طلال,regular
2,28832968,name,Khawlah Bint Al 'Azwar Street,regular
3,28833349,name,شارع الشهيد,regular
4,28833525,name,King Abdallah I street,regular


In [12]:
# Questionable data entry 4

query = '''
    SELECT 
        *
    FROM nodes_tags 
    WHERE key = 'phone'
    ;'''
show_result(query, 5, 'Questionable data entry 4')

Questionable data entry 4


,id,key,value,type
0,474956673,phone,+962 6 460 3000,regular
1,697379357,phone,+96264657777,regular
2,974451473,phone,+96265002347,regular
3,975955978,phone,0096264614161,regular
4,1076180988,phone,+962-6-4629317,regular


In [13]:
# Closing the connection with the database

c.close() # closing the cursor
db.close() # closing the connection

## 8. Data Problems 

After processing this map's data, here are some noticeable issues: 

 
- The quality of the data is questionable, as shown in 'Questionable data entry 1', a school is tagged as a bank. 
- Some of the entries have mixed languages (containing Arabic and English) in the names and other values, as seen in 'Questionable data entry 2' 
- There are inconsistencies in providing names in different languages. For example, some of the tags have keys named "name" which could be in Arabic or English (not standardized) as seen in 'Questionable data entry 3'. Then it might (or might not) have other keys in the same tag named "name:ar" (for Arabic names), "name:en" (for English names). In addition, some of the names are provided in only one language and some have 2, 3, 4, or even much more languages. 
- Some of the names that are marked Arabic are actually Turkish or in some other languages! 
- Country and city codes for phone numbers are not consistent as seen in 'Questionable data entry 4'. Some have + to represent the 00, some have spaces, and some have "-" separating the the city codes from the phone number. 


---------------


## 9. Data Improvements and Conclusion

After investigating the dataset and searching for the top contributers, [Lyx](http://wiki.openstreetmap.org/wiki/User:Lyx), I found that there is a [Project Jordan](http://wiki.openstreetmap.org/wiki/WikiProject_Jordan) for the OpenStreetMap. 

In there there is a good structure for contributing to the dataset: 
    
    name=عمان
    name:ar=عمان
    name:en=Amman

Here we understand that the "name" should always in Arabic, then there is the Arabic name, English name, and any other languages. The problem was in not following the guidelines properly. So in the "name" field some users used a mixed language. We need to make it clear to users not to use mixed languages. 

To avoid mistakes in phone numbers, we could have the country and city codes restricted, and give some guidelines on the phone number formatting. 

I learned that the dataset of Amman is relatively small, although it's a big city (7x bigger than San Francisco size with 3x more people). In addition, the data is mostly outdated because as we can see from the user contributions query above, the majority of the contributions were in years 2012, and 2013.


---------------

## 10. Resources:

- [1]: [Mapzen - Amman Map Data](https://mapzen.com/data/metro-extracts/metro/amman_jordan/)
- [2]: [Amman, Jordan's Wikipedia Page](https://en.wikipedia.org/wiki/Amman)
- [3]: [Rich Output in Jypyter](https://nbviewer.jupyter.org/github/ipython/ipython/blob/master/examples/IPython%20Kernel/Rich%20Output.ipynb)
- [4]: [jupyter contrib nbextensions](https://github.com/ipython-contrib/jupyter_contrib_nbextensions)
- [5]: [WikiProject Jordan](http://wiki.openstreetmap.org/wiki/WikiProject_Jordan)
- [6]: [Working with SQLite Databases using Python and Pandas](https://www.dataquest.io/blog/python-pandas-databases/)
- [7]: [Regular expression with utf8 issue](http://stackoverflow.com/questions/10546442/python-regular-expression-with-utf8-issue)
- [8]: [Working with large CSV files in Python](http://pythondata.com/working-large-csv-files-python/)